*Instructions*
---------------
Before starting to work, please extract data from data_02.zip to a folder called **data** located at the same level that this notebook.

# 2) Loading and saving data

In [ ]:
import pandas as pd

## Loading CSV

First, we need to prepare one csv file to read!

In [ ]:
! ls ./data #not working on windows

In [ ]:
! unzip ./data/T100_SEGMENT_ALL_CARRIER_2015.zip #not working on windows

In [ ]:
! ls -lrt ./

In [ ]:
#Defining path to CSV file
my_file_path='914310910_T_T100_SEGMENT_ALL_CARRIER_2015_All.csv'

In [ ]:
#Reading file and saving in df
df = pd.read_csv(my_file_path)
df.head()

In [ ]:
#df?

In [ ]:
#Number of rows on Dataframe
len(df)

Make some operations with the df

In [ ]:
df[df['DEPARTURES_PERFORMED'] > 50]['PAYLOAD'].sum()

In [ ]:
%%timeit
df[df['DEPARTURES_PERFORMED'] > 50]['PAYLOAD'].sum()

In [ ]:
len(df[df['DEPARTURES_PERFORMED'] > 50])

In order to download a file using Python, the use of requests module is recommended

In [ ]:
#!wget -o http://datos.madrid.es/egob/catalogo/207831-0-accidentes-trafico.xls

### Example:

In [ ]:
import requests
#url="http://datos.madrid.es/egob/catalogo/210104-180-multas-circulacion-detalle.csv"
url = 'http://climatedataapi.worldbank.org/climateweb/rest/v1/country/cru/tas/year/CAN.csv'
response = requests.get(url)
if response.status_code != 200:
    print('Failed to get data:', response.status_code)
else:
    print('First 100 characters of data are')
    print(response.text[:100])

In [ ]:
import requests
my_path="http://datos.madrid.es/egob/catalogo/210104-180-multas-circulacion-detalle.csv"
multas_response = requests.get(my_path)

In [ ]:
multas_path = 'multas.csv'
multas_file = open(multas_path, 'wb')
multas_file.write(multas_response.content)
multas_file.close()

In [ ]:
! wc -l {multas_path} #unix

In [ ]:
! head -n 5 {multas_path} #unix

Con el encoding latin-1 nos ahorramos problemas con caracteres acentuados.

In [ ]:
multas_path2="data/201712_multas_circulacion_detalle.csv"
df_multas = pd.read_csv(multas_path2, sep=';', encoding='latin-1')
df_multas.head(20)

Strings can be treaky...

In [ ]:

df_multas[df_multas['LUGAR'] == ' LUIS DE HOYOS SAINZ FTE 196 ']

but with a bit of care...

In [ ]:
mi_portal ="LUIS DE HOYOS SAINZ FTE 196".strip()

In [ ]:
df_multas[df_multas['LUGAR'].str.strip() == mi_portal]

We can play with this dataframe... XD

In [ ]:
df_multas[df_multas['LUGAR'].str.startswith("ARDEMANS")]

In [ ]:
df_multas['IMP_BOL'].mean()

In [ ]:
df_multas['IMP_BOL'].sum()

In [ ]:
descuentos = df_multas['DESCUENTO']
descuentos.value_counts()

#### Exercise

Calculate the amount that the city hall took in, after discount. Assume that the discount is 50%, for each line that has 'SI' in column 'DESCUENTO'.

In [ ]:
df_multas['DESCUENTO_PCT'] = 0.5 *(df_multas['DESCUENTO'] == 'SI')
df_multas['DESCUENTO_PCT'].head()

In [ ]:
df_multas['IMPORTE_FINAL'] = df_multas['IMP_BOL'] * (1 - df_multas['DESCUENTO_PCT'])

#### Let's time it

In [ ]:
df_multas = pd.read_csv(multas_path2, sep=';', encoding='latin-1')

In [ ]:
%%timeit
df_multas['DESCUENTO_PCT'] = .5 *(df_multas['DESCUENTO'] == 'SI')
df_multas['IMPORTE_FINAL'] = df_multas['IMP_BOL'] * (1 - df_multas['DESCUENTO_PCT'])

# Saving to Excel

In order to save and read excels, pandas needs addtional modules: xlrd, xlwt and xlsxwriter or openpyxl

In [ ]:
df[df['DEPARTURES_PERFORMED'] > 50].to_excel('out.xls')

In [ ]:
pd.ExcelWriter?

In [ ]:
df[df['DEPARTURES_PERFORMED'] > 50].to_excel('out.xlsx')

# Saving to CSV

In [ ]:
df.to_csv('out.csv')

If we don't pass any argument, to_csv will return the contents of the DataFrame, in csv format, as a string.

In [ ]:
file_as_string = df.to_csv()

file_as_string[:100]

# To Sql Database

In [ ]:
import sqlite3
conn = sqlite3.connect('example.db')

In [ ]:
df.to_sql('traffic', conn)

# To dictionary and to json

json is the most widely used data interchange format over the web.

In [ ]:
!pwd #only for unix

In [ ]:
df[:3].to_dict()

In [ ]:
df[:3].to_dict(orient='rows')

In [ ]:
df[:3].to_json()

# Reading Excel

In [ ]:
df2 = pd.read_excel('out.xls')
df2

In [ ]:
import requests

response = requests.get('http://datos.madrid.es/egob/catalogo/207831-0-accidentes-trafico.xls')
out = open('accidentes.xls', 'wb')
out.write(response.content)
out.close()

In [ ]:
accidentes = pd.read_excel('data/01_ ACCIDENTES POR TIPO EN  DISTRITOS.xls')
accidentes

In [ ]:
file_path = 'data/01_ ACCIDENTES POR TIPO EN  DISTRITOS.xls'

accidentes_2010 = pd.read_excel(file_path, 
                                index_col=0, 
                                header=7, 
                                sheet='2010', 
                                skip_footer=1,
                                usecols='A:K')
accidentes_2010

In [ ]:
all_accidents = pd.read_excel(file_path, index_col=0, header=7, sheet_name=None)
all_accidents['2009']

In [ ]:
for k, v in all_accidents.items():
    print(v)

#### Exercise (homework)

Consolidate the excel into one DataFrame: you will need to create a 'YEAR' column. Think how you can iterate through all the DataFrames. Think also where you can take the value of 'YEAR' from.

# Reading mysql database

In [ ]:
pd.read_sql_query('SELECT * FROM traffic WHERE SEATS > 160', conn)